In [ ]:
!pip install torch opencv-python
!pip install git+https://github.com/ultralytics/yolov5


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import torch
import cv2
from pathlib import Path
import os

def detect_and_save_results(image_path, output_image_path, output_txt_path):
    # Load pre-trained YOLOv5 model
    model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

    img = cv2.imread(image_path)

    results = model(img)

    detections = results.xyxy[0].cpu().numpy()  # [x_min, y_min, x_max, y_max, confidence, class_id]

    output_image_dir = Path(output_image_path).parent
    output_txt_dir = Path(output_txt_path).parent
    output_image_dir.mkdir(parents=True, exist_ok=True)
    output_txt_dir.mkdir(parents=True, exist_ok=True)

    with open(output_txt_path, 'w') as f:
        for detection in detections:
            x_min, y_min, x_max, y_max, conf, class_id = detection
            label = model.names[int(class_id)]
            f.write(f'{label} {conf:.2f} {x_min:.0f} {y_min:.0f} {x_max:.0f} {y_max:.0f}\n')

            cv2.rectangle(img, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (255, 0, 0), 2)
            cv2.putText(img, f'{label} {conf:.2f}', (int(x_min), int(y_min) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    cv2.imwrite(output_image_path, img)

image_folder = '/content/drive/My Drive/all images'
output_folder = '/content/drive/My Drive/satellite_output'

os.makedirs(output_folder, exist_ok=True)

for image_filename in os.listdir(image_folder):
    if image_filename.endswith('.jpg') or image_filename.endswith('.png'):
        image_path = os.path.join(image_folder, image_filename)
        output_image_path = os.path.join(output_folder, f"output_{image_filename}")
        output_txt_path = os.path.join(output_folder, f"output_{image_filename.split('.')[0]}.txt")

        detect_and_save_results(image_path, output_image_path, output_txt_path)

print(f"Detection complete. Results saved to {output_folder}.")
